伪代码：
```
def greedy_algorithm(problem):
    # 1. 初始化：可能包括排序、创建结果容器等
    solution = []
    # 2. 遍历所有可选项，通常需要先按某种规则排序
    for item in sorted(problem.items, key=贪心策略规则):
        # 3. 贪心选择：如果当前选择满足条件，就采纳它
        if is_feasible(solution, item):
            solution.append(item)
    # 4. 返回最终构造的解
    return solution
```

# 经典问题与代码示例

### 示例 1：找零钱问题（Coin Change）

问题描述：假设硬币体系为 [100, 50, 20, 10, 5, 1]（单位：分），需要找零 amount 分，求使用硬币的最少数量。

贪心策略：每次选择面值不超过剩余金额的最大硬币。对于人民币体系，这个策略是有效的。

In [3]:
def coin_change_greedy(coins, amount):
    """
    使用贪心算法解决找零钱问题（针对特定硬币体系）
    :param coins: list[int], 硬币面值列表，应降序排列
    :param amount: int, 需要找零的总金额
    :return: list[int], 使用的硬币面值列表
    """
    coins.sort(reverse=True) # 确保硬币按面值从大到小排序
    result = [] # 用于存放找零的硬币
    remaining = amount # 剩余需要找零的金额

    for coin in coins:
        # 当当前硬币面值小于等于剩余金额时，就尽可能多地使用它
        while coin <= remaining:
            result.append(coin)
            remaining -= coin # 从剩余金额中扣除当前硬币面值

    # 检查是否正好找开
    if remaining == 0:
        return result
    else:
        # 对于无法正好找开的情况（虽然在此硬币体系下不会发生）
        return None

In [4]:
# 测试数据
test_coins = [100, 50, 20, 10, 5, 1]
test_amount = 186  # 1元8角6分

change = coin_change_greedy(test_coins, test_amount)
print(f"找零 {test_amount} 分，需要硬币：{change}")
print(f"硬币数量：{len(change)} 枚")
# 输出：找零 186 分，需要硬币：[100, 50, 20, 10, 5, 1]
# 硬币数量：6 枚

找零 186 分，需要硬币：[100, 50, 20, 10, 5, 1]
硬币数量：6 枚


### 示例 2：区间调度问题（Interval Scheduling）

问题描述：给你很多个会议（每个会议有开始和结束时间），如何安排才能使你在一个会议室里能举行的会议数量最多？

贪心策略：每次选择结束时间最早的会议。这样可以为后面留下更多的时间。

In [12]:
def interval_scheduling(intervals):
    """
    解决最大不相交区间问题（区间调度）
    :param intervals: list of [start, end], 区间列表
    :return: list of [start, end], 被选中的区间列表
    """
    # 1. 按结束时间顺序排序
    intervals_sorted = sorted(intervals, key=lambda x: x[1])

    selected = [] # 存放被选中的区间
    last_end_time = -float('inf') # 初始化上一个选中区间的结束时间

    for interval in intervals_sorted:
        start, end = interval
        # 2. 贪心选择：如果当前区间的开始时间不早于上一个选中区间的结束时间，则选择它
        if start >= last_end_time:
            selected.append(interval)
            last_end_time = end # 更新最后结束时间
    
    return selected

In [13]:
# 测试数据：每个子列表代表一个会议 [开始时间, 结束时间]
test_intervals = [
    [1, 3], [2, 4], [3, 5], [4, 6], [5, 7],
    [1, 2], [2, 3], [1, 4]
]

result = interval_scheduling(test_intervals)
print("最多可以安排的会议（按结束时间排序后）：")
for meeting in result:
    print(f"  会议 [{meeting[0]}, {meeting[1]}]")
print(f"总计 {len(result)} 个会议")

最多可以安排的会议（按结束时间排序后）：
  会议 [1, 2]
  会议 [2, 3]
  会议 [3, 5]
  会议 [5, 7]
总计 4 个会议


### 示例 3：霍夫曼编码（Huffman Coding） - 概念与简化实现

霍夫曼编码是一种用于无损数据压缩的贪心算法。其核心思想是：给出现频率高的字符分配较短的编码，出现频率低的字符分配较长的编码。

贪心策略：反复将频率最小的两个节点合并，构建一棵二叉树。

由于完整实现较长，这里展示其核心的贪心合并过程：

In [1]:
import heapq

In [2]:
class Node:
    """霍夫曼树的节点类"""
    def __init__(self, char, freq):
        self.char = char # 字符（叶子节点才有）
        self.freq = freq # 频率
        self.left = None
        self.right = None
    
    # 为了能在堆中比较，定义比较方法
    def __lt__(self, other):
        return self.freq < other.freq

In [3]:
def build_huffman_tree(char_freq):
    """
    构建霍夫曼树
    :param char_freq: dict, 字符及其频率，如 {'a': 5, 'b': 9, 'c': 12, 'd': 13, 'e': 16, 'f': 45}
    :return: Node, 霍夫曼树的根节点
    """
    # 1. 初始化：为每个字符创建叶子节点，并放入最小堆（优先队列）
    heap = [Node(char, freq) for char, freq in char_freq.items()]
    heapq.heapify(heap) # 将列表转换为最小堆

    # 2. 贪心合并：当堆中不止一个节点时
    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)

        # 创建一个新的内部节点，其频率为子节点频率之和
        merged = Node(None, left.freq + right.freq)
        merged.left = left
        merged.right = right

        # 将新节点推回堆中
        heapq.heappush(heap, merged)

    # 堆中剩下的最后一个节点就是霍夫曼树的根节点
    return heap[0] if heap else None

In [4]:
# 测试数据
test_freq = {'a': 5, 'b': 9, 'c': 12, 'd': 13, 'e': 16, 'f': 45}
root = build_huffman_tree(test_freq)
print("霍夫曼树构建完成！根节点频率 =", root.freq if root else 0)
# 输出：霍夫曼树构建完成！根节点频率 = 100

霍夫曼树构建完成！根节点频率 = 100


# 实践练习

### 练习 1：跳跃游戏

问题：给定一个非负整数数组 nums，你最初位于数组的第一个下标。数组中的每个元素代表你在该位置可以跳跃的最大长度。判断你是否能够到达最后一个下标。 示例：nums = [2,3,1,1,4]，返回 True（从下标 0 跳 1 步到下标 1，再跳 3 步到最后一个下标）。 贪心策略提示：不是模拟每一步跳多远，而是维护一个 最远可到达位置。

### 练习 2：分发饼干

问题：有一群孩子和一堆饼干，每个孩子有一个胃口值 g[i]，每块饼干有一个尺寸 s[j]。只有饼干尺寸 >= 孩子胃口时，孩子才能满足。求最多能满足多少个孩子。 示例：g = [1,2,3], s = [1,1]，输出 1。 贪心策略提示：为了满足更多孩子，应该用最小的饼干去满足胃口最小的孩子（或者反过来思考）。

In [33]:
def distribute(g, s):
    g.sort(); s.sort()
    satisfied = 0

    for i in range(len(g)):
        if len(s) == 0:
            break
        else:
            while len(s) > 0 and s[0] < g[i]:
                s.pop(0)
            if len(s) == 0:
                break
            else:
                s.pop(0)
                satisfied += 1
                continue

    return satisfied

In [37]:
g = [1, 2, 3]; s = [1, 1, 3, 4, 8]
distribute(g, s)

3

### 练习 3：买卖股票的最佳时机 II

问题：给定一个数组 prices，其中 prices[i] 是一支给定股票第 i 天的价格。你可以无限次地完成交易（即买卖一支股票），但你不能同时参与多笔交易（必须在再次购买前出售掉之前的股票）。计算你能获得的最大利润。 示例：prices = [7,1,5,3,6,4]，输出 7（在 1 买 5 卖，利润 4；在 3 买 6 卖，利润 3；总利润 7）。 贪心策略提示：利润可以分解为每天的差价。只要 今天的价格比昨天高，就把这部分差价算作利润。